In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

In [2]:
import time
import sys
import os
import random
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms, utils
import numpy as np
from tqdm.auto import tqdm
from collections import Counter
from matplotlib import pyplot as plt
from collections import OrderedDict

SEED = 0
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
import pandas as pd

In [4]:
import wandb

# start a new experiment
wandb.init(project="cs-th-en-seq2seq_attention")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wannaphong. Use `wandb login --relogin` to force relogin
2022-05-23 04:50:56.214628: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [5]:
path_dataset = os.path.join('..','dataset','cs')

train_filepaths = os.path.join(path_dataset,'train.tsv')
dev_filepaths = os.path.join(path_dataset,'dev.tsv')
test_filepaths = os.path.join(path_dataset,'test.tsv')

train_df = pd.read_csv(train_filepaths,sep="\t")
dev_df = pd.read_csv(dev_filepaths,sep="\t")
test_df = pd.read_csv(test_filepaths,sep="\t")

In [6]:
def load_data(data_path):
    input_texts = [str(i) for i in list(data_path['word'])]
    target_texts =[str(i) for i in  list(data_path['roman'])]
    return input_texts, target_texts

In [7]:
# Define special characters
UNK_token = '<UNK>'
PAD_token = '<PAD>'
START_token = '<start>'
END_token = '<end>'
MAX_LENGTH = 60

In [8]:
class Language:
    def __init__(self, name,char2index={},index2char={}, is_input=False):
        self.name = name
        self.characters = set()
        self.n_chars = 0
        self.char2index = {}
        self.index2char = {}

        if is_input == True:
            self.index2char = { 0: PAD_token, 1: UNK_token, 2: START_token, 3: END_token }
            self.char2index = { ch:i for i, ch in self.index2char.items() } #reverse dictionary
            self.n_chars = 4
        else:
            self.index2char = { 0: PAD_token, 1: START_token, 2: END_token }
            self.char2index = { ch:i for i, ch in self.index2char.items() } #reverse dictionary
            self.n_chars = 3
        if char2index != {} and index2char != {}:
            print("cat!!!")
            self.char2index = char2index
            self.index2char = index2char
            self.characters = set(list(self.char2index.keys()))

    def addText(self, text):
        for character in text:
            self.addCharacter(character)
    
    def addCharacter(self, character):
        if character not in self.char2index.keys():
            self.char2index[character] = self.n_chars
            self.index2char[self.n_chars] = character
            self.n_chars += 1

In [9]:
def indexesFromText(lang, text):
    """returns indexes for all character given the text in the specified language"""
    return [lang.char2index[char] for char in text]

def tensorFromText(lang, text):
    """construct a tensor given the text in the specified language"""
    indexes = indexesFromText(lang, text)
    indexes.append(lang.char2index[END_token])
    
    no_padded_seq_length = len(indexes) # Number of characters in the text (including <END> token)
    # Add padding token to make all tensors in the same length
    for i in range(len(indexes), MAX_LENGTH): # padding
        indexes.append(lang.char2index[PAD_token])
        
    return torch.tensor(indexes, dtype=torch.long), no_padded_seq_length

def filterPair(p1, p2):
    """filter for the pair the both texts has length less than `MAX_LENGTH`"""
    return len(p1) < MAX_LENGTH and len(p2) < MAX_LENGTH

def tensorsFromPair(pair, lang1, lang2):
    """construct two tensors from a pair of source and target text specified by source and target language"""
    input_tensor, input_length = tensorFromText(lang1, pair[0])
    target_tensor, target_length = tensorFromText(lang2, pair[1])
    return input_tensor, target_tensor, input_length, target_length



class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        input_text, target_text, lang_th, lang_th_romanized = sample['input_text'], sample['target_text'],                                                               sample['lang_th'], sample['lang_th_romanized']

        input_tensor, target_tensor, input_length, target_length = tensorsFromPair([input_text, target_text], 
                                                                                   lang_th, 
                                                                                   lang_th_romanized)
        
        return {
                'input_text': input_text,
                'target_text': target_text,
                'input_length': input_length,
                'target_length': target_length,
                'input_tensor': input_tensor,
                'target_tensor': target_tensor
               }

In [10]:
class ThaiRomanizationDataset(Dataset):
    """Thai Romanization Dataset class"""
    def __init__(self, 
                 data_path,
                 config = {},
                 transform=transforms.Compose([ ToTensor() ])):

        input_texts, target_texts = load_data(data_path)
        
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.transform = transform
        self.config=config
        self.lang_th = Language('th', self.config['char_to_ix'],self.config['ix_to_char'],is_input=True)
        self.lang_th_romanized = Language('th_romanized', self.config['target_char_to_ix'],self.config['ix_to_target_char'], is_input=False)
        self.counter = Counter()
        self.pairs = []
        self.prepareData()

    def prepareData(self):
        for i in range(len(self.input_texts)):
            
            input_text = str(self.input_texts[i])
            target_text = str(self.target_texts[i])
            
            # Count the number of input and target sequences with length `x`
            self.counter.update({ 
                                  'len_input_{}'.format(len(input_text)): 1, 
                                  'len_target_{}'.format(len(target_text)): 1 
                                })
            
            if filterPair(input_text, target_text):
                self.pairs.append((input_text, target_text))
                self.lang_th.addText(input_text)
                self.lang_th_romanized.addText(target_text)    

    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        
        sample = dict()
        sample['input_text'] = self.pairs[idx][0]
        sample['target_text'] = self.pairs[idx][1]
        
        sample['lang_th'] = self.lang_th
        sample['lang_th_romanized'] = self.lang_th_romanized

        if self.transform:
            sample = self.transform(sample)

        return sample

In [11]:
def load_model(model_path):
    return torch.load(model_path)

In [12]:
data=load_model("thai2rom-pytorch-72-Copy1.attn.v6.best_epoch-72.tar")

In [13]:
data['ix_to_target_char']

{0: '<PAD>',
 1: '<start>',
 2: '<end>',
 3: 'k',
 4: 'a',
 5: 's',
 6: 'e',
 7: 'm',
 8: ' ',
 9: 'b',
 10: 'u',
 11: 'n',
 12: 'h',
 13: 'i',
 14: 'r',
 15: 'o',
 16: 't',
 17: 'p',
 18: 'g',
 19: 'w',
 20: 'l',
 21: 'd',
 22: 'c',
 23: 'y',
 24: 'f',
 25: '1',
 26: '(',
 27: ')',
 28: '2',
 29: '5',
 30: '3',
 31: '6',
 32: '4',
 33: '7',
 34: '0',
 35: '!',
 36: '9',
 37: '8',
 38: '"'}

In [14]:
thai_romanization_dataset = ThaiRomanizationDataset(train_df,config=data)

cat!!!
cat!!!


In [15]:
print(thai_romanization_dataset.lang_th_romanized.index2char)

{0: '<PAD>', 1: '<start>', 2: '<end>', 3: 'v', 4: 'x', 5: 'z', 6: 'é', 7: 'q', 8: "'", 9: 'j', 10: 'ฺ', 11: 'è', 12: '岸', 13: '田', 14: '文', 15: '雄', 16: '.', 17: '–', 18: 'g', 19: 'w', 20: 'l', 21: 'd', 22: 'c', 23: 'y', 24: 'f', 25: '1', 26: '(', 27: ')', 28: '2', 29: '5', 30: '3', 31: '6', 32: '4', 33: '7', 34: '0', 35: '!', 36: '9', 37: '8', 38: '"'}


In [16]:
SEED = 0
BATCH_SIZE = 256

TRAIN_RATIO = 0.8

N = len(thai_romanization_dataset)

print('Number of samples: ', N)

Number of samples:  3041


In [17]:
train_dataset_loader = torch.utils.data.DataLoader(
                                             thai_romanization_dataset,
                                             batch_size=BATCH_SIZE, 
                                             num_workers=0)

In [18]:
print('Number of train mini-batches', len(train_dataset_loader))

Number of train mini-batches 12


In [19]:
# ## Seq2Seq Model architecture

# ## 1. Encoder

# Encoder 
#     - Embedding layer :(vocaburay_size, embedding_size) 
#         Input: (batch_size, sequence_length)
#         Output: (batch_size, sequence_length, embebeding_size)
#       
#     - Bi-LSTM layer : (input_size, hidden_size, num_layers, batch_first=True)
#         Input: (input=(batch_size, seq_len, embebeding_size),  hidden)
#         Output: (output=(batch_size, seq_len, hidden_size),
#                  (h_n, c_n))
#      
#      
# __Steps:__
# 
# 1. Receives a batch of source sequences (batch_size, MAX_LENGTH) and a 1-D array of the length for each sequence (batch_size).
#      
# 2. Sort sequences in the batch by sequence length (number of tokens in the sequence where <PAD> token is excluded).
# 
# 3. Feed the batch of sorted sequences into the Embedding Layer to maps source character indices into vectors. (batch_size,  sequence_length, embebeding_size)
# 
# 4. Use `pack_padded_sequence` to let LSTM packed input with same length at time step $t$ together. This will reduce time required for training by avoid feeding `<PAD>` token to the LSTMs.
# 
# 
# 5. Returns LSTM outputs in the unsorted order, and the LSTM hidden state vectors.
#      

# In[19]:




class Encoder(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_size, hidden_size, dropout=0.5):
        """Constructor"""
        super(Encoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.character_embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.lstm = nn.LSTM(input_size=embedding_size, 
                            hidden_size=hidden_size // 2, 
                            bidirectional=True,
                            batch_first=True)
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, sequences, sequences_lengths):
        batch_size = sequences.size(0)
        self.hidden = self.init_hidden(batch_size) # batch_size

        # sequences :(batch_size, sequence_length=MAX_LENGTH)
        # sequences_lengths: (batch_size)  # an 1-D indicating length of each sequence (excluded <PAD> token) in `seq`
        
        # 1. Firstly we sort `sequences_lengths` according to theirs values and keep list of indexes to perform sorting
        sequences_lengths = np.sort(sequences_lengths)[::-1] # sort in ascending order and reverse it
        index_sorted = np.argsort(-sequences_lengths) # use negation in sort in descending order
        index_unsort = np.argsort(index_sorted) # to unsorted sequence
        
        
        # 2. Then, we change position of sequence in `sequences` according to `index_sorted`
        index_sorted = torch.from_numpy(index_sorted)
        sequences = sequences.index_select(0, index_sorted)
        
        # 3. Feed to Embedding Layer
        
        sequences = self.character_embedding(sequences)
        sequences = self.dropout(sequences)
        
#         print('sequences',sequences.size(), sequences)
            
        # 3. Use function: pack_padded_sequence to let LSTM packed input with same length at time step T together
        
        # Quick fix: Use seq_len.copy(), instead of seq_len to fix `Torch.from_numpy not support negative strides`
        # ndarray.copy() will alocate new memory for numpy array which make it normal, I mean the stride is not negative any more.

        sequences_packed = nn.utils.rnn.pack_padded_sequence(sequences,
                                                             sequences_lengths.copy(),
                                                             batch_first=True)
#         print('sequences_packed', sequences_packed)

        # 4. Feed to LSTM
        sequences_output, self.hidden = self.lstm(sequences_packed, self.hidden)
        
        # 5. Unpack
        sequences_output, _ = nn.utils.rnn.pad_packed_sequence(sequences_output, batch_first=True)

        # 6. Un-sort by length
        index_unsort = torch.from_numpy(index_unsort)
        sequences_output = sequences_output.index_select(0, Variable(index_unsort))

#         print('hidden shape', self.hidden[0].shape, self.hidden[0], self.hidden[1].shape, self.hidden[1])
        return sequences_output, self.hidden
    
    def init_hidden(self, batch_size):
        h_0 = torch.zeros([2, batch_size, self.hidden_size // 2], requires_grad=True)
        c_0 = torch.zeros([2, batch_size, self.hidden_size // 2], requires_grad=True)
        
        return (h_0, c_0)
    
def save_model(name, epoch, loss, model):
    print('Save model at epoch ', epoch)
    torch.save({
        'epoch': epoch,
        'loss': loss,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'char_to_ix': thai_romanization_dataset.lang_th.char2index,
        'ix_to_char': thai_romanization_dataset.lang_th.index2char,
        'target_char_to_ix': thai_romanization_dataset.lang_th_romanized.char2index,
        'ix_to_target_char':thai_romanization_dataset.lang_th_romanized.index2char,
        'encoder_params': (INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT), 
        'decoder_params': (OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT)
        
    }, "{}.best_epoch-{}.tar".format(name, epoch))
    
 
    

       


# ## Decoder

#    
# Decoder architecture
# 
#     - Embedding layer :(vocabulary_size, embebeding_size)
#         Input: (batch_size, sequence_length=1)
#         Output: (batch_size, sequence_length=1, embebeding_size)
#     - RNN layer :input_size=embebeding_size, hidden_size, num_layers, batch_first=True)
#         Input: (input=(batch_size, input_size=embedding_dimension), hidden:tuple=encoder_hidden
#         Output: (batch_size, seq_len, hidden_size), (h_n, c_n)
#     - Attention Layer: (in_features=hidden_size, out_features=hidden_size, bias=True)
#     - Linear Layer: (in_features, out_features=vocabulary_size)
#         Input: (batch_size, hidden_size)
#         Output: (batch_size, vocabulary_size)
#     
#     - Softmax layer
#         Input: (batch_size, vocabulary_size)
#         Output: (batch_size, vocabulary_size)
# 
# 
# 
# For the Attention mechanishm in the Decoder, Luong-style attention [[Luong et. al (2015)](https://arxiv.org/abs/1508.04025)] is used. 
# 
# 
# 
# __Steps:__
# 
# 1. Receives a batch of <start> token (batch_size, 1) and a batch of Encoder's hidden state.
#      
# 2. Embed input into vectors.
# 
# 3. Feed vectors from (2) to the LSTM.
# 
# 4. Feed the output of LSTM at time step $t_1$ and Encoder output to the Attention Layer.
# 
# 5. Attention layer, returns weights for Encoder's hidden states in every time step (masked out the time step with <PAD> token), then multiply with Encoder's hidden states to obtain a context vector
#     
# 6. Concatenate both decoder hidden state and the context vector, feed to a linear layer, and return its output.
# 
# 7. Decoder then returns, final output, decoder's hidden state, attention weights, and context vector at time step $t$

# In[20]:



class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()

        self.method = method
        self.hidden_size = hidden_size

        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.other = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs, mask):
        # hidden: B x 1 x h ; 
        # encoder_outputs: B x S x h

        # Calculate energies for each encoder output
        if self.method == 'dot':
            attn_energies = torch.bmm(encoder_outputs, hidden.transpose(1, 2)).squeeze(2)  # B x S
        elif self.method == 'general':
            attn_energies = self.attn(encoder_outputs.view(-1, encoder_outputs.size(-1)))  # (B * S) x h
            attn_energies = torch.bmm(attn_energies.view(*encoder_outputs.size()),
                                      hidden.transpose(1, 2)).squeeze(2)  # B x S
        elif self.method == 'concat':
            attn_energies = self.attn(
                torch.cat((hidden.expand(*encoder_outputs.size()), encoder_outputs), 2))  # B x S x h
            attn_energies = torch.bmm(attn_energies,
                                      self.other.unsqueeze(0).expand(*hidden.size()).transpose(1, 2)).squeeze(2)

        attn_energies = attn_energies.masked_fill(mask == 0, -1e10)

        # Normalize energies to weights in range 0 to 1
        return F.softmax(attn_energies, 1)

class AttentionDecoder(nn.Module): 
    
    def __init__(self, vocabulary_size, embedding_size, hidden_size, dropout=0.5):
        """Constructor"""
        super(AttentionDecoder, self).__init__()
        self.vocabulary_size = vocabulary_size
        self.hidden_size = hidden_size
        self.character_embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.lstm = nn.LSTM(input_size=embedding_size + self.hidden_size,
                            hidden_size=hidden_size,
                            bidirectional=False,
                            batch_first=True)
        
        self.attn = Attn(method="general", hidden_size=self.hidden_size)
        self.linear = nn.Linear(hidden_size * 2, vocabulary_size)
        
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, input, last_hidden, last_context, encoder_outputs, mask):
        """"Defines the forward computation of the decoder"""
        # input: (B, 1) ,
        # last_hidden: (num_layers * num_directions, B, hidden_dim)
        # last_context: (B, 1, hidden_dim)
        # encoder_outputs: (B, S, hidden_dim)
        
        embedded = self.character_embedding(input)
        embedded = self.dropout(embedded)
        
        # embedded: (batch_size, emb_dim)
        rnn_input = torch.cat((embedded, last_context), 2)

        output, hidden = self.lstm(rnn_input, last_hidden)        
        attn_weights = self.attn(output, encoder_outputs, mask)  # B x S
    
        #  context = (B, 1, S) x (B, S, hidden_dim)
        #  context = (B, 1, hidden_dim)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)  
        
        output = torch.cat((context.squeeze(1), output.squeeze(1)), 1)
        output = self.linear(output)
        
        return output, hidden, context, attn_weights


# ## Seq2Seq model
# 
# This class encapsulate _Decoder_ and _Encoder_ class.
# 
# __Steps:__
# 
# 1. The input sequcence $X$ is fed into the encoder to receive one hidden state vector.
# 
# 2. The initial decoder hidden state is set to be the hidden state vector of the encoder
# 
# 3. Add a batch of `<start>` tokens (batch_size, 1) as the first input $y_1$
#     
# 4. Then, decode within a loop:
#     - Inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector $z$ into the decoder
#     - Receiveing a prediction $\hat{y}$ and a new hidden state $s_t$
#     - Then, either use teacher forcing to let groundtruth target character as the input for the decoder at time step $t+1$, or let the result from decoder as the input for the next time step.

# In[21]:


class Seq2Seq(nn.Module): 

    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = 0

        assert encoder.hidden_size == decoder.hidden_size
    
    def create_mask(self, source_seq):
        mask = (source_seq != self.pad_idx)
        return mask
        
  
    def forward(self, source_seq, source_seq_len, target_seq, teacher_forcing_ratio = 0.5):
        """
            Parameters:
                - source_seq: (batch_size x MAX_LENGTH) 
                - source_seq_len: (batch_size x 1)
                - target_seq: (batch_size x MAX_LENGTH)

            Returns
        """
        batch_size = source_seq.size(0)
        start_token = thai_romanization_dataset.lang_th_romanized.char2index["<start>"]
        end_token = thai_romanization_dataset.lang_th_romanized.char2index["<end>"]
        max_len = MAX_LENGTH
        target_vocab_size = self.decoder.vocabulary_size

        # init a tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, target_vocab_size)
        
        if target_seq is None:
            assert teacher_forcing_ratio == 0, "Must be zero during inference"
            inference = True
        else:
            inference = False

    
        # feed mini-batch source sequences into the `Encoder`
        encoder_outputs, encoder_hidden = self.encoder(source_seq, source_seq_len)

        # create a Tensor of first input for the decoder
        decoder_input = torch.tensor([[start_token] * batch_size]).view(batch_size, 1)
        
        # Initiate decoder output as the last state encoder's hidden state
        decoder_hidden_0 = torch.cat([encoder_hidden[0][0], encoder_hidden[0][1]], dim=1).unsqueeze(dim=0)
        decoder_hidden_1 = torch.cat([encoder_hidden[1][0], encoder_hidden[1][1]], dim=1).unsqueeze(dim=0)
        decoder_hidden = (decoder_hidden_0, decoder_hidden_1) # (hidden state, cell state)

        # define a context vector
        decoder_context = Variable(torch.zeros(encoder_outputs.size(0), encoder_outputs.size(2))).unsqueeze(1)
        
        max_source_len = encoder_outputs.size(1)
        mask = self.create_mask(source_seq[:, 0:max_source_len])
            
       
        for di in range(max_len):
            decoder_output, decoder_hidden, decoder_context, attn_weights = self.decoder(decoder_input,
                                                                                    decoder_hidden,
                                                                                    decoder_context,
                                                                                    encoder_outputs,
                                                                                    mask)
            # decoder_output: (batch_size, target_vocab_size)

            topv, topi = decoder_output.topk(1)
            outputs[di] = decoder_output
    
            teacher_force = random.random() < teacher_forcing_ratio


            decoder_input = target_seq[:, di].reshape(batch_size, 1) if teacher_force else topi.detach() 

            if inference and decoder_input == end_token:
                return outputs[:di]
        return outputs

In [20]:
INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT = data['encoder_params']
OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT = data['decoder_params']

_encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM)
_decoder = AttentionDecoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM)

#model = Seq2Seq(_encoder, _decoder)
model = Seq2Seq(_encoder, _decoder)
model.load_state_dict(data['model_state_dict'])

<All keys matched successfully>

In [21]:
data

{'epoch': 72,
 'loss': 0.260413283491746,
 'model_state_dict': OrderedDict([('encoder.character_embedding.weight',
               tensor([[-0.0726,  0.1101, -0.0081,  ..., -0.3005, -0.1339,  0.0756],
                       [ 0.0037, -0.0155, -0.0141,  ...,  0.0126,  0.0087,  0.0058],
                       [ 0.0094,  0.0075,  0.0122,  ...,  0.0100, -0.0125, -0.0116],
                       ...,
                       [ 0.0368, -0.0111,  0.3473,  ..., -0.0964, -0.0316,  0.0479],
                       [ 0.1193, -0.3317,  0.0783,  ..., -0.1472,  0.1345, -0.0841],
                       [ 0.1311,  0.0516, -0.1785,  ..., -0.2887, -0.0801, -0.0385]])),
              ('encoder.lstm.weight_ih_l0',
               tensor([[-0.2214,  0.1812, -0.4905,  ...,  0.7163, -0.1723, -0.3168],
                       [-0.5070, -0.2123,  0.0631,  ...,  0.3151, -0.0470, -0.0560],
                       [-0.3759,  0.0631, -0.0918,  ...,  0.1992, -0.2342,  0.1362],
                       ...,
                 

In [22]:
data.keys()

dict_keys(['epoch', 'loss', 'model_state_dict', 'optimizer_state_dict', 'char_to_ix', 'ix_to_char', 'target_char_to_ix', 'ix_to_target_char', 'encoder_params', 'decoder_params'])

In [23]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(data['optimizer_state_dict'])
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [24]:
char_to_ix = data['char_to_ix']
ix_to_char = data['ix_to_char'] 
target_char_to_ix = data['target_char_to_ix']
ix_to_target_char = data['ix_to_target_char']

In [25]:
from tqdm.auto import tqdm

In [26]:
N_EPOCHS = 5

In [27]:
wandb.config = {"learning_rate": learning_rate, "epochs": N_EPOCHS, "batch_size": BATCH_SIZE}

In [28]:
wandb.watch(model, log_freq=100)

[]

In [29]:
def train(model, iterator, optimizer, criterion, clip, teacher_forcing_ratio=0.5):
    
    model.train()
    
    epoch_loss = 0
    for i, batch in tqdm(enumerate(iterator), total = len(iterator)):
        optimizer.zero_grad()

        source_seq, source_seq_len = batch['input_tensor'], batch['input_length']
        batch_size = source_seq.size(0)
        
        # target_seq: (batch_size , MAX_LENGTH)
        # output: (MAX_LENGTH , batch_size , target_vocab_size)
        target_seq = batch['target_tensor']

        output = model(source_seq, source_seq_len, target_seq, teacher_forcing_ratio=teacher_forcing_ratio)
        
        # target_seq -> (MAX_LENGTH , batch_size)
        target_seq = target_seq.transpose(0, 1)

        # target_seq -> ((MAX_LENGTH - 1) * batch_size)
        target_seq = target_seq[1:].contiguous().view(-1)

        # output -> ((MAX_LENGTH -1) * batch_size, target_vocab_size)        
        output = output[1:].view(-1, output.shape[-1])

        loss = criterion(output, target_seq)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        wandb.log({"train_loss":loss.item()})
        
        epoch_loss += loss.item()
        
        
    return epoch_loss / len(iterator)



In [30]:
char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char = thai_romanization_dataset.lang_th.char2index ,thai_romanization_dataset.lang_th.index2char , thai_romanization_dataset.lang_th_romanized.char2index ,  thai_romanization_dataset.lang_th_romanized.index2char

In [31]:
def evaluate(model, iterator, criterion):
    
    model.eval()

    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            source_seq, source_seq_len = batch['input_tensor'], batch['input_length']
            batch_size = source_seq.size(0)

            # target_seq: (batch_size , MAX_LENGTH)
            # output: (MAX_LENGTH , batch_size , target_vocab_size)
            target_seq = batch['target_tensor']
            output = model(source_seq, source_seq_len, target_seq)
        
            # target_seq -> (MAX_LENGTH , batch_size)
            target_seq = target_seq.transpose(0, 1)

            # target_seq -> ((MAX_LENGTH - 1) * batch_size)
            target_seq = target_seq[1:].contiguous().view(-1)

            # output -> ((MAX_LENGTH -1) * batch_size, target_vocab_size)        
            output = output[1:].view(-1, output.shape[-1])

            loss = criterion(output, target_seq)
            wandb.log({"evaluate_loss":loss.item()})
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [32]:
def inference(model, text, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char):
    model.eval()

    input_seq =  [ch for ch in text] +  ['<end>']
    numericalized = [char_2_ix[ch] for ch in input_seq] 
    
#     print('input ',numericalized)
    sentence_length = [len(numericalized)]

    tensor = torch.LongTensor(numericalized).view(1, -1)
    
#     print(tensor)
    translation_tensor_logits = model(tensor, sentence_length, None, 0) 
#     print(translation_tensor_logits)
    try:
        translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1).cpu().numpy()
        translation_indices = [t for t in translation_tensor]
        
#         print('translation_tensor', translation_tensor)
        translation = [ix_to_target_char[t] for t in translation_tensor]
    except:
        translation_indices = [0]
        translation = ['<pad>']
    return ''.join(translation), translation_indices

def show_inference_example(model, input_texts, target_texts, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char):
    for index, input_text in enumerate(input_texts):
        prediction, indices = inference(model, input_text, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char)
        print('groundtruth: {}'.format(target_texts[index]))
        print(' prediction: {} {}\n'.format(prediction, indices))
        

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [33]:
show_inference_example(model, ['การ'], [''], char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char)

groundtruth: 
 prediction: 8xè [37, 4, 11]



In [34]:
CLIP = 5

best_valid_loss = float('inf')

In [35]:
# for epoch in tqdm(range(1, N_EPOCHS+1)):
#  model.train()
#  start_time = time.time()
#  train_loss = train(model, train_dataset_loader, optimizer, criterion, CLIP, teacher_forcing_ratio=0.5)
#  #valid_loss = evaluate(model, val_dataset_loader, criterion)
#  end_time = time.time()
#  save_model('new_model/thai2rom-pytorch-%s.attn.v6' % str(epoch), epoch, train_loss, model)
#  wandb.log({"epoch": epoch,"epoch_train_loss": train_loss,"epoch_time":end_time-start_time})

In [36]:
# epoch

In [37]:
# save_model('new_model/thai2rom-pytorch-%s.attn.v6' % str(epoch), epoch, train_loss, model)

In [38]:
# wandb.log({"epoch": epoch,"epoch_train_loss": train_loss,"epoch_time":end_time-start_time})

In [39]:
for epoch in tqdm(range(1, N_EPOCHS+1)):
 model.train()
 start_time = time.time()
 train_loss = train(model, train_dataset_loader, optimizer, criterion, CLIP, teacher_forcing_ratio=0.5)
 #valid_loss = evaluate(model, val_dataset_loader, criterion)
 end_time = time.time()
 save_model('model/thai2rom-pytorch-%s.attn.v6' % str(epoch), epoch, train_loss, model)
 wandb.log({"epoch": epoch,"epoch_train_loss": train_loss,"epoch_time":end_time-start_time})

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Save model at epoch  1


  0%|          | 0/12 [00:00<?, ?it/s]

Save model at epoch  2


...

Save model at epoch  3


  0%|          | 0/12 [00:00<?, ?it/s]

Save model at epoch  4


  0%|          | 0/12 [00:00<?, ?it/s]

Save model at epoch  5


In [40]:
# for epoch in tqdm(range(6,10+1)):
#  model.train()
#  start_time = time.time()
#  train_loss = train(model, train_dataset_loader, optimizer, criterion, CLIP, teacher_forcing_ratio=0.5)
#  #valid_loss = evaluate(model, val_dataset_loader, criterion)
#  end_time = time.time()
#  save_model('new_model/thai2rom-pytorch-%s.attn.v6' % str(epoch), epoch, train_loss, model)
#  wandb.log({"epoch": epoch,"epoch_train_loss": train_loss,"epoch_time":end_time-start_time})